In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import math

from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from datetime import date,datetime

Using TensorFlow backend.


In [2]:
DATA_DIR = Path('.', 'data', 'final', 'public')
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')
test_values = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.33, random_state=42)

In [4]:
X_train.describe()

geo_level_1_id  geo_level_2_id  geo_level_3_id  count_floors_pre_eq  \
count   174602.000000   174602.000000   174602.000000        174602.000000   
mean        13.895454      701.410935     6261.399423             2.129254   
std          8.024348      412.469421     3648.427229             0.728312   
min          0.000000        0.000000        1.000000             1.000000   
25%          7.000000      350.000000     3073.000000             2.000000   
50%         12.000000      706.000000     6271.000000             2.000000   
75%         21.000000     1050.000000     9414.000000             2.000000   
max         30.000000     1427.000000    12567.000000             7.000000   

                 age  area_percentage  height_percentage  \
count  174602.000000    174602.000000      174602.000000   
mean       26.437899         8.009416           5.431261   
std        72.711965         4.394350           1.914698   
min         0.000000         1.000000           2.000000   
25%        10.000000         5.000000           4.000000   
50%        15.000000         7.000000           5.000000   
75%        30.000000         9.000000           6.000000   
max       995.000000        96.000000          32.000000   

       has_superstructure_adobe_mud  has_superstructure_mud_mortar_stone  \
count                 174602.000000                        174602.000000   
mean                       0.088613                             0.761589   
std                        0.284185                             0.426113   
min                        0.000000                             0.000000   
25%                        0.000000                             1.000000   
50%                        0.000000                             1.000000   
75%                        0.000000                             1.000000   
max                        1.000000                             1.000000   

       has_superstructure_stone_flag  ...  has_secondary_use_agriculture  \
count                  174602.000000  ...                  174602.000000   
mean                        0.034461  ...                       0.064558   
std                         0.182411  ...                       0.245745   
min                         0.000000  ...                       0.000000   
25%                         0.000000  ...                       0.000000   
50%                         0.000000  ...                       0.000000   
75%                         0.000000  ...                       0.000000   
max                         1.000000  ...                       1.000000   

       has_secondary_use_hotel  has_secondary_use_rental  \
count            174602.000000             174602.000000   
mean                  0.033321                  0.007932   
std                   0.179475                  0.088710   
min                   0.000000                  0.000000   
25%                   0.000000                  0.000000   
50%                   0.000000                  0.000000   
75%                   0.000000                  0.000000   
max                   1.000000                  1.000000   

       has_secondary_use_institution  has_secondary_use_school  \
count                  174602.000000             174602.000000   
mean                        0.001008                  0.000338   
std                         0.031733                  0.018379   
min                         0.000000                  0.000000   
25%                         0.000000                  0.000000   
50%                         0.000000                  0.000000   
75%                         0.000000                  0.000000   
max                         1.000000                  1.000000   

       has_secondary_use_industry  has_secondary_use_health_post  \
count               174602.000000                  174602.000000   
mean                     0.001163                       0.000183   
std                      0.034078                    

In [30]:
if 'presotere' in globals():
    del presotere
class presotere(TransformerMixin,BaseEstimator):
    
    def __init__(self,caso,geocode,scaler):
        self.lencoder_col=list([])
        self.object_cols=list([])
        self.number_cols = list([])
        self.lencoder_col=list([])
        self._caso=caso
        self.legeo = LabelEncoder()
        self.lencoder= list([])
        self._sns_data=None
        self._y=None
        self._geocode=geocode
        self._scaler=scaler
        
    def inicializamodelo(self,caso):

        if caso == 1:
            self.legeo = LabelEncoder()
            self.lencoder= list([])
        if caso == 2:
            self.legeo = LabelEncoder()
            self.lencoder= OneHotEncoder(handle_unknown='ignore', sparse=False)   
        
        
    def transform(self,X,y=None, **kwargs):
        print("Transformando ",self.get_params())
        if self._caso == 1:
            contador=0
            self._sns_data=X[self.number_cols].copy()
            for col in X[self.object_cols].columns:
                self.lencoder.append(LabelEncoder())
                self.lencoder[contador].fit(X[col])
                self._sns_data[col]=self.lencoder[contador].transform(X[col])  
                contador=contador+1
                
                
        if self._caso == 2:  
            nada = None
            self._sns_data=None
            nada = self.lencoder.fit_transform(X[self.object_cols])
            co1c=0
            self.lencoder_col=list([])
            for co1 in self.lencoder.categories_:
                for co2 in co1:
                    self.lencoder_col.append(self.object_cols[co1c]+"_"+co2)
                co1c=co1c+1
            objedf=pd.DataFrame(nada,columns=self.lencoder_col,
                                         index=X[self.object_cols].index.tolist())
            self._sns_data=pd.concat([X[self.number_cols].copy(),objedf],axis=1)  

                
        if self._geocode>0:        
            geo_level_1_fact=math.pow(10,int(math.log(self._sns_data['geo_level_2_id'].max(),10)+1))
            geo_level_2_fact=math.pow(10,int(math.log(self._sns_data['geo_level_3_id'].max(),10)+1))
            self._sns_data['geo_level_n']=  self._sns_data['geo_level_1_id']*geo_level_1_fact*geo_level_2_fact+self._sns_data['geo_level_2_id']*geo_level_2_fact+self._sns_data['geo_level_3_id']
            self._sns_data['geo_level']=self._sns_data['geo_level_n']#.astype(np.int64).astype(str)
            self._sns_data['geo_level_2']=self._sns_data['geo_level_n']*self._sns_data['geo_level_n']
            self._sns_data['geo_level_cod']=self._sns_data['geo_level_n'].astype(np.int64)


            
            self._sns_data=self._sns_data.drop(['geo_level_n','geo_level'],axis=1)

#tratamiento de age            
#            self._sns_data['age1']=self._sns_data['age'].apply(lambda x:x  if x <200 else 200)
#            self._sns_data['age1']=self._sns_data['age'].apply(lambda x:x  if x <200 else 200)
#        self._sns_data['age2']=np.log(self._sns_data['age']+1)
#        self._sns_data['age2']=(self._sns_data['age']*self._sns_data['age'])
#        self._sns_data=self._sns_data.drop(['age'],axis=1)
#            self._sns_data['age2']=self._sns_data['age'].apply(lambda x:0  if x <200 else 1)
#            self._sns_data=self._sns_data.drop(['age','age1'],axis=1)
######            
        self._sns_data=self._sns_data.drop(['count_floors_pre_eq'],axis=1)
        if self._scaler=="MinMax":
            mi_scaler = MinMaxScaler()
        if self._scaler=="Standard":
            mi_scaler = StandardScaler()            
        self._sns_data[list(self._sns_data.columns)]=mi_scaler.fit_transform(self._sns_data.values.astype(float))
        lcolum_x = list(self._sns_data.columns) 
        return self._sns_data
    
    def fit(self,X, y=None, **kwargs):
        for key, value in kwargs.items():
            if key=="caso":
                self._caso=value
            if key=="geocode":
                self._geocode=value
            if key=="scaler":
                self._scaler=value
        s = (X.dtypes == 'object')
        self.object_cols = list(s[s].index)
        s = (X.dtypes != 'object')        
        self.number_cols = list(s[s].index) 
        self._y=y
        self.inicializamodelo(self._caso)
        self.transform(X,y, **kwargs)
        return self

    def predict(self, X):
        return(self._y)

    def fit_transform(self,X, y=None, **kwargs):
            miX=X.values
            miX=miX.reshape(-1, 1)
            self.fit(X, y, **kwargs)
            nada=self.transform(X, **kwargs)
            return(nada)
        
    def set_params(self,**kwargs):
            for key, value in kwargs.items():
                if key=="caso":
                    self._caso=value
                if key=="geocode":
                    self._geocode=value
                if key=="scaler":
                    self._scaler=value                                 
            return self    
        
    def get_params(self,**kwargs):
        return({"caso":self._caso,"geocode":self._geocode,"scaler":self._scaler})
    

In [31]:
def cargamodelo(modelcaso):
    global model
    if modelcaso==1:
        model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial',max_iter=500)
    if modelcaso==2:    
        model = RandomForestRegressor(max_depth=20, random_state=0,n_estimators=100)
    if modelcaso==3:
        model = DecisionTreeClassifier(random_state=0,max_depth=20) 
    if modelcaso==4:
        model = MultinomialNB
    if modelcaso==5:
        model = DecisionTreeRegressor(random_state=0)   
    if modelcaso==6:
        model = SVC(gamma='auto',verbose=True,kernel='linear', probability=True)       
    if modelcaso==7:
        model = RandomForestClassifier(max_depth=20, random_state=0,n_estimators=100)     
    if modelcaso==80:
        model = XGBClassifier()   
    if modelcaso==81:        
        model = XGBRegressor()        
    if modelcaso==9:    
        model = Sequential()
        model.add(Dense(12, input_dim=36, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(15, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      

In [32]:
class mimodelo(RandomForestRegressor):
    
    def fit(self, X, y,sample_weight=None):
#        yfinal=np.log(y)
        yfinal=y
        return(super().fit(X,yfinal,sample_weight))
        
        
    def predict(self, X):
#        return(np.exp(super().predict(X)).round().astype("int64"))
        return(super().predict(X).round().astype("int64"))

In [33]:
if 'pipeline' in globals():    
    del pipeline
final=Pipeline(memory=None,
     steps=[('Preprosessor', presotere(caso=1, geocode=1, scaler='MinMax')), ('Modelo', mimodelo(bootstrap=True, criterion='mse', max_depth=27, max_features='auto',
     max_leaf_nodes=None, min_impurity_decrease=0.0,
     min_impurity_split=None, min_samples_leaf=2, min_samples_split=20,
     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
     oob_score=False, random_state=0, verbose=1, warm_start=False))])


In [34]:
final.fit(X_train,y_train)
f1=f1_score(y_test, final.predict(X_test), average='micro') 
print("Score",f1)

Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}
Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   28.8s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  1.0min finished


Transformando  {'caso': 1, 'geocode': 1, 'scaler': 'MinMax'}


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s


Score 0.7369736857405319


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished


In [35]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body,  
    attribs = body.attributes;
var command = "theNotebook = " + "'"+attribs['data-notebook-name'].value+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [36]:
f=open(DATA_DIR / 'resultados.txt','a+')
print(theNotebook,file=f)
print(final.steps,file=f)
print("Score:",  f1,file=f)
print(datetime.now().strftime("%d/%m/%Y %H:%M:%S"),file=f)
f.close()